In [104]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ubuntu/varios/skforecast


In [105]:
import numpy as np
from numba import njit
import numba as nb
import pandas as pd


@njit
def ewma(
    x: np.ndarray, alpha: float, adjust: bool = True, ignore_na: bool = False
) -> np.ndarray:
    """
    Calculate the Exponentially Weighted Moving Average of an xay. This
    implementation mimics the pandas implementation of the same function.

    This implementation is based on the work of rjenc29 in the following
    link: https://github.com/rjenc29/numerical/blob/master/utilities/ewma.py

    Parameters
    ----------
    x : np.xay
        The xay to calculate the EWMA.
    alpha : float
        The decay factor.
    adjust : bool, optional
        Adjust the weights for the bias towards zero. The default is False.
    ignore_na : bool, optional
        Ignore missing values. The default is False.

    Returns
    -------
    np.xay
        The EWMA of the
    """

    old_wt_factor = 1.0 - alpha
    new_wt = 1.0 if adjust else alpha

    n = x.shape[0]
    output = np.empty(n)

    weighted_avg = x[0]
    is_observation = weighted_avg == weighted_avg
    nobs = int(is_observation)
    output[0] = weighted_avg if (nobs >= 1) else np.nan
    old_wt = 1.0

    for i in range(1, n):
        cur = x[i]
        is_observation = cur == cur
        nobs += int(is_observation)
        if weighted_avg == weighted_avg:
            if is_observation or (not ignore_na):
                old_wt *= old_wt_factor
                if is_observation:
                    if weighted_avg != cur:
                        weighted_avg = (
                            (old_wt * weighted_avg) + (new_wt * cur)
                        ) / (old_wt + new_wt)
                    if adjust:
                        old_wt += new_wt
                    else:
                        old_wt = 1.0
        elif is_observation:
            weighted_avg = cur

        output[i] = weighted_avg if (nobs >= 1) else np.nan

    return output

In [106]:
# Example usage
rng = np.random.default_rng(42)
x = rng.random(50)
series = pd.Series(x)
alpha = 0.5  # The alpha value for the ewma

smoothed_x = ewma(x, alpha, adjust=True)
print(smoothed_x)
assert np.allclose(smoothed_x, series.ewm(alpha=alpha, adjust=True).mean())


[0.77395605 0.55057098 0.72658637 0.71100326 0.3926415  0.68875876
 0.72523419 0.75576852 0.44132693 0.44586086 0.40831111 0.66760135
 0.65573179 0.7392518  0.59132848 0.40928083 0.48193336 0.27287451
 0.55025337 0.59095892 0.67452337 0.51452463 0.74261136 0.81786624
 0.79812487 0.49638178 0.48155139 0.26267758 0.20848354 0.44576624
 0.5952642  0.78138697 0.55360616 0.46203293 0.46579437 0.32763287
 0.22877719 0.35224106 0.2895752  0.4796946  0.45842326 0.64555073
 0.67290791 0.49263728 0.66244854 0.73360645 0.56054241 0.42443526
 0.55346538 0.34660893]


In [107]:
%%timeit -n 1000 -r 10
ewma(x, alpha)

98.8 μs ± 67 μs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [108]:
%%timeit -n 1000 -r 10
series.ewm(alpha=alpha, adjust=True).mean()

130 μs ± 2.94 μs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [109]:
import numpy as np

def exponentially_weighted_mean(x, alpha):
    if not (0 < alpha <= 1):
        raise ValueError("Alpha should be in the range (0, 1].")
    
    n = len(x)
    weights = np.array([(1 - alpha) ** i for i in range(n)])
    weights = weights[::-1]  # Reverse to match array's order
    
    ewm = np.sum(x * weights) / np.sum(weights)
    return ewm


def exponentially_weighted_mean_2(x, alpha):
    if not (0 < alpha <= 1):
        raise ValueError("Alpha should be in the range (0, 1].")
    
    n = len(x)
    weights = (1 - alpha) ** np.arange(n)[::-1]  # Generate and reverse weights
    sum_weights = (1 - (1 - alpha) ** n) / alpha  # Sum of geometric series
    
    ewm = np.sum(x * weights) / sum_weights
    return ewm


@njit  # This tells Numba to use the fastest mode
def exponentially_weighted_mean_3(x:np.ndarray, alpha:float):
    """
    Calculate the exponentially weighted mean of an array.

    Parameters
    ----------
    x : numpy.ndarray
        The input array.
    alpha : float
        The decay factor.

    Returns
    -------
    float
        The exponentially weighted mean.


    """
    if not (0 < alpha <= 1):
        raise ValueError("Alpha should be in the range (0, 1].")
    
    n = len(x)
    weights = 0
    sum_weights = 0
    for i in range(n):
        weight = (1 - alpha) ** (n - 1 - i)
        weights += x[i] * weight
        sum_weights += weight

    return weights / sum_weights

In [110]:
print(exponentially_weighted_mean(x, alpha))
assert np.allclose(exponentially_weighted_mean(x, alpha), series.ewm(alpha=alpha, adjust=True).mean().iloc[-1])

0.3466089325170127


In [111]:
print(exponentially_weighted_mean_2(x, alpha))
assert np.allclose(exponentially_weighted_mean_2(x, alpha), series.ewm(alpha=alpha, adjust=True).mean().iloc[-1])

0.3466089325170127


In [112]:
print(exponentially_weighted_mean_3(x, alpha))
assert np.allclose(exponentially_weighted_mean_3(x, alpha), series.ewm(alpha=alpha, adjust=True).mean().iloc[-1])

0.34660893251701275


In [113]:
%%timeit
exponentially_weighted_mean(x, alpha)

25.8 μs ± 407 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [114]:
%%timeit
exponentially_weighted_mean_2(x, alpha)

15.5 μs ± 4.75 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [115]:
%%timeit
exponentially_weighted_mean_3(x, alpha)

806 ns ± 1.51 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [116]:
exponentially_weighted_mean_3(series.head(6).to_numpy(), alpha)

0.6887587564882396

In [117]:
series.rolling(window=5).apply(lambda x: exponentially_weighted_mean_3(x.to_numpy(), alpha))

0          NaN
1          NaN
2          NaN
3          NaN
4     0.392641
5     0.687385
6     0.729460
7     0.756592
8     0.433171
9     0.447524
10    0.399406
11    0.665757
12    0.652517
13    0.748844
14    0.596016
15    0.409312
16    0.475946
17    0.260526
18    0.544157
19    0.590947
20    0.683079
21    0.515576
22    0.757764
23    0.826499
24    0.804808
25    0.490635
26    0.480488
27    0.247196
28    0.188826
29    0.434400
30    0.598454
31    0.791059
32    0.562991
33    0.470212
34    0.466440
35    0.319000
36    0.210951
37    0.345745
38    0.284012
39    0.480143
40    0.462642
41    0.658995
42    0.683252
43    0.499188
44    0.668344
45    0.742483
46    0.557800
47    0.416420
48    0.555428
49    0.336421
dtype: float64